# `System.Text.Json` with F#

The most unpopular way to deserialize types in F# is to explicitly express the intent to traverse the document looking for values. My other studies of JSON in F# in [this directory](../json) show the more popular ways to get this done.

I am so devoted to expressing explicit traversal that I built [a module of functions](https://github.com/BryanWilhite/Songhay.Modules/blob/853c780e1f4aa278426d924b429f0f45c7950f9a/Songhay.Modules/JsonDocumentUtility.fs) dedicated to this miserable task:

In [1]:
#!fsharp

open System.Linq
open System.Text.Json

/// <summary>
/// Wraps <see cref="JsonException" /> property
/// in <see cref="Error" />.
/// </summary>
/// <param name="elementName">The <see cref="JsonElement" /> name.</param>
let resultError (elementName: string) =
    Error(JsonException $"the expected `{elementName}` element is not here.")

/// <summary>
/// Tries to return the <see cref="JsonElement" /> property
/// of the specified <see cref="JsonElement" /> object.
/// </summary>
/// <param name="elementName">The <see cref="JsonElement" /> name.</param>
/// <param name="element">The <see cref="JsonElement" />.</param>
let tryGetProperty (elementName: string) (element: JsonElement) =
    match element.TryGetProperty elementName with
    | false, _ -> resultError elementName
    | true, el -> Ok el

## the importance of `tryGetProperty`

The `tryGetProperty` function shown above is a functional wrapper around `JsonElement.TryGetProperty` [ 📖[docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonelement.trygetproperty?view=net-7.0)], formally expressing that trying to get a “child” property will always return `Result<JsonElement,JsonException>` where the exception is centralized by `resultError`.

The following JSON has a single `top` “child” property:

In [2]:
#!fsharp

let jDoc = JsonDocument.Parse(@"
    {
        ""top"": {
            ""one"": ""this is first"",
            ""two"": ""this is second"",
            ""three"": {
                ""p1"": ""this is three-point-one"",
                ""p2"": ""this is three-point-two""
            }
        }
    }
")

We can enumerate the objects of the `JsonDocument.RootElement` to see this lone property:

In [3]:
#!fsharp

jDoc.RootElement.EnumerateObject()

index value 0 "top": {\n "one": "this is first",\n "two": "this is second",\n "three": {\n "p1": "this is three-point-one",\n "p2": "this is three-point-two"\n }\n } Value {
 "one": "this is first",
 "two": "this is second",
 "three": {
 "p1": "this is three-point-one",
 "p2": "this is three-point-two"
 }
 } one "this is first" two "this is second" three {
 "p1": "this is three-point-one",
 "p2": "this is three-point-two"
 } p1 "this is three-point-one" p2 "this is three-point-two" Name top

The name `RootElement` has been quite confusing for me because I _must_ remember (even on a rainy day) that a JSON root is not an XML root. Every XML document has _one_ root element while (of course) a JSON document can have _multiple_ child elements (or properties) like the following:

In [4]:
#!fsharp

let jDoc = JsonDocument.Parse(@"
    {
        ""top"": {
            ""one"": ""this is first"",
            ""two"": ""this is second"",
            ""three"": {
                ""p1"": ""this is three-point-one"",
                ""p2"": ""this is three-point-two""
            }
        },
        ""bottom"": {
            ""is-footer"": false
        }
    }
")

My `tryGetProperty` function is important because it allows me to, say, traverse `jDoc` to get the children of `three`:

In [5]:
#!fsharp

open System.Linq

let result =
    jDoc.RootElement
    |> tryGetProperty "top"
    |> Result.bind (tryGetProperty "three")
    |> Result.map (fun element -> element.EnumerateObject().ToArray())

result |> Result.defaultValue [||]

index value 0 "p1": "this is three-point-one" Value "this is three-point-one" Name p1 1 "p2": "this is three-point-two" Value "this is three-point-two" Name p2

## why not just serialize JSON exclusively?

As of this writing, I am still attracted to JSON document traversal instead of defining a type and serializing.

[Ian Griffiths](https://www.linkedin.com/in/idg10/) supports my investment in two ways—first, is this quote:

>You might find this a little less convenient than conventional serialization, in which we define a .NET type whose structure resembles the JSON we expect to see, and then use JsonSerializer.Deserialize to convert the JSON into an instance of that type. But if you're familiar with these APIs (or you've watched and read the video and blogs linked to above) you'll know that the approach shown here places a much lower load on the garbage collector, and that it has the potential to provide much higher throughput in high-performance systems.
>
>—[Ian Griffiths](https://endjin.com/blog/2024/02/dotnet-jsonelement-parse-errors)
>

Second, is his ‘up-sell’ to `Corvus.JsonSchema` \[🔗 [GitHub](https://github.com/corvus-dotnet/Corvus.JsonSchema/#corvusjsonschema) \]

@[BryanWilhite](https://twitter.com/BryanWilhite)
